In [ ]:
# This notebook demonstrates using bankroll to load positions across brokers
# and highlights some basic portfolio rebalancing opportunities based on a set of desired allocations.
#
# The default portfolio allocation is described (with comments) in notebooks/Rebalance.example.ini.
# Copy this to Rebalance.ini in the top level folder, then edit accordingly, to provide your own
# desired allocation.

%cd ..
import pandas as pd
from bankroll import *
from configparser import ConfigParser
from decimal import Decimal
from functools import reduce
from ib_insync import IB, util
from itertools import *
from math import *
import logging
import operator
import re

In [ ]:
util.startLoop()

accounts = AccountAggregator.fromSettings(AccountAggregator.allSettings(), lenient=False)
assert accounts.marketDataProvider

In [ ]:
stockPositions = [p for p in accounts.positions() if isinstance(p.instrument, Stock)]
stockPositions.sort(key=lambda p: p.instrument)

In [ ]:
values = liveValuesForPositions(stockPositions, accounts.marketDataProvider)

In [ ]:
config = ConfigParser(interpolation=None)

try:
    config.read_file(open('Rebalance.ini'))
except OSError:
    config.read_file(open('notebooks/Rebalance.example.ini'))

In [ ]:
def parsePercentage(str):
    match = re.match(r'([0-9\.]+)%', str)
    if match:
        return float(match[1]) / 100
    else:
        return float(str)

In [ ]:
settings = config['Settings']

In [ ]:
ignoredSecurities = {s.strip() for s in settings['ignored securities'].split(',')}

In [ ]:
maximumDeviation = parsePercentage(settings['maximum deviation'])

In [ ]:
baseCurrency = Currency[settings['base currency']]

In [ ]:
categoryAllocations = {category: parsePercentage(allocation) for category, allocation in config['Portfolio'].items()}

In [ ]:
totalAllocation = sum(categoryAllocations.values())
assert abs(totalAllocation - 1) < 0.0001, f'Category allocations do not total 100%, got {totalAllocation:.2%}'

In [ ]:
securityAllocations = {}
for category, categoryAllocation in categoryAllocations.items():
    securities = {security.upper(): parsePercentage(allocation) for security, allocation in config[category].items()}
    
    totalAllocation = sum(securities.values())
    assert abs(totalAllocation - 1) < 0.0001, f'Allocations in category {category} do not total 100%, got {totalAllocation:.2%}'
    
    securityAllocations.update({security: allocation * categoryAllocation for security, allocation in securities.items()})

In [ ]:
cashBalance = accounts.balance()
cashBalance

In [ ]:
portfolioValue = reduce(operator.add, (value for p, value in values.items() if p.instrument.symbol not in ignoredSecurities), cashBalance)
portfolioValue

In [ ]:
def color_deviations(val):
    color = 'black'
    if abs(val) > maximumDeviation:
        if val > 0:
            color = 'green'
        else:
            color = 'red'
            
    return f'color: {color}'

In [ ]:
def positionPctOfPortfolio(position) -> float:
    if position not in values:
        return nan
    
    return float(values[position].quantity) / float(portfolioValue.cash[position.instrument.currency].quantity)

In [ ]:
rows = {p.instrument.symbol: [
    p.quantity,
    values.get(p, nan),
    positionPctOfPortfolio(p),
    securityAllocations.get(p.instrument.symbol),
    positionPctOfPortfolio(p) - securityAllocations.get(p.instrument.symbol, 0),
] for p in stockPositions if p.instrument.symbol not in ignoredSecurities}

In [ ]:
missing = {symbol: [
    nan,
    nan,
    nan,
    allocation,
    nan,
] for symbol, allocation in securityAllocations.items() if symbol not in rows}

In [ ]:
df = pd.DataFrame.from_dict(data=dict(chain(rows.items(), missing.items())), orient='index', columns=[
    'Quantity',
    'Market value',
    '% of portfolio',
    'Desired %',
    'Deviation'
]).sort_index()

df.style.format({
    'Quantity': '{:.2f}',
    '% of portfolio': '{:.2%}',
    'Desired %': '{:.2%}',
    'Deviation': '{:.2%}'
}).applymap(color_deviations, 'Deviation').highlight_null()